In [63]:
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib
import pandas as pd
import re
from io import BytesIO
from collections import Counter

In [51]:
StrToReplaceBrand = "%brandName%"
urlpage = "https://www.leparking-moto.fr/moto-occasion/.html#!/moto-occasion/"+StrToReplaceBrand+".html"
hdrs = {'User-Agent': 'Mozilla / 5.0 (X11 Linux x86_64) AppleWebKit / 537.36 (KHTML, like Gecko) Chrome / 52.0.2743.116 Safari / 537.36'}    

urls = []
for brandName in ["honda","yamaha","kawasaki","suzuki", "benelli" ]:#"royal-enfield", "moto-guzzi"]:#"kawazaki"
    urls.append( (urlpage.replace(StrToReplaceBrand, brandName), brandName ) )
CH_DIR = 'C:\\Users\\avuat\\Python Projects\\Motorcycle Classifier'
os.chdir(CH_DIR)
os.getcwdb()

print(urls)

[('https://www.leparking-moto.fr/moto-occasion/.html#!/moto-occasion/honda.html', 'honda'), ('https://www.leparking-moto.fr/moto-occasion/.html#!/moto-occasion/yamaha.html', 'yamaha'), ('https://www.leparking-moto.fr/moto-occasion/.html#!/moto-occasion/kawasaki.html', 'kawasaki'), ('https://www.leparking-moto.fr/moto-occasion/.html#!/moto-occasion/suzuki.html', 'suzuki'), ('https://www.leparking-moto.fr/moto-occasion/.html#!/moto-occasion/benelli.html', 'benelli')]


In [48]:
#Install driver
opts=webdriver.ChromeOptions()
opts.headless=True

driver_brand = webdriver.Chrome(ChromeDriverManager().install() ,options=opts)
driver_ad = webdriver.Chrome(ChromeDriverManager().install() ,options=opts)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/95.0.4638.54/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\avuat\.wdm\drivers\chromedriver\win32\95.0.4638.54]
C:\Users\avuat\AppData\Local\Temp/ipykernel_5452/2307519185.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_brand = webdriver.Chrome(ChromeDriverManager().install() ,options=opts)
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\avuat\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
C:\Users\avuat\AppData\Local\Temp/ipykernel_5452/2307519185.py:6: DeprecationWarning:

In [49]:
def saveImage(brand, title, src):
    if src is None or src == "":
        return 0
    brandFolder = os.path.join(".","images", brand)
    if not os.path.exists(brandFolder):
        os.makedirs(brandFolder)
    modelFolder = os.path.join(brandFolder, title)
    if not os.path.exists(modelFolder):
        os.makedirs(modelFolder)
    localName = os.path.join(modelFolder, src.split("/")[-1])
    r = requests.get(src)
    i = Image.open(BytesIO(r.content))
    filename, file_extension = os.path.splitext(localName)
    if file_extension not in [".jpeg", ".png",".jpg"]:
        print(file_extension, " not in allowed formats")
        return 0
    i.save(localName)
    return localName

CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, ' ', raw_html)
    return cleantext

In [53]:
dataPictures = []

for url, brandName in urls:
    driver_brand.get(url)
    print("Brand : "+brandName)
    try:
        for i in range(10,80):
            print("Going page : "+str(i))
            driver_brand.execute_script("ctrl.set_pageReload("+str(i)+")")
            resultAd = [a.get_attribute('href') for a in  driver_brand.find_elements(By.XPATH, "//ul[@id='resultats']//section[@class='clearfix complete-holder']//a[@class='external tag_f_titre']") ]
            blockTitleListBrand = [b.get_attribute("innerHTML") for b in driver_brand.find_elements(By.XPATH, "//section[@class='clearfix complete-holder']//div[@class='block-title-list']//span[@class='title-block brand']")]
            blockTitleListModel = [cleanhtml(m.get_attribute("innerHTML")).replace("ABS","").lower().strip(" ") for m in driver_brand.find_elements(By.XPATH, "//section[@class='clearfix complete-holder']//div[@class='block-title-list']//span[@class='sub-title title-block'][1]")]
            print("Found : "+str(len(resultAd)) + " bikes, with " +str(len(blockTitleListBrand))+ " brands and "+str(len(blockTitleListModel))+ " models")
            if len(resultAd) == 0:
                print("No more results for "+brandName)
                continue
            for idx, res in enumerate(resultAd):
                brandFormatted = blockTitleListBrand[idx]
                modelFormatted = blockTitleListModel[idx]
                print("Looking at a nice "+str(brandFormatted)+ " "+str(modelFormatted)+ " ... ")
                driver_ad.get(res)
                img = driver_ad.find_elements(By.XPATH, "//div[@class='fiche-descri clearfix']//img")
                if len(img) == 1:
                    print("Image found !")
                    img = img[0]
                else :
                    print("No image found for "+ str(href))
                    continue
                title = img.get_attribute('title')
                src = img.get_attribute('src')
                if "visuel_generique.jpg" in src:
                    print("Default image")
                    continue
                cylindre = re.sub('[^0-9]+', '', modelFormatted)
                words = re.sub('[^a-zA-Z ]+', '', modelFormatted).split()
                modelFull = " ".join(sorted(set(words), key=words.index)) 
                pathImage = saveImage(brandName,modelFull , src)
                year =  re.sub('[^0-9]', '', [cleanhtml(m.get_attribute("innerHTML")) for m in driver_ad.find_elements(By.XPATH, "//div[@class='detail-descri']//ul[@class='list-info-detail clearfix']//li[1]")][0])
                print("Image saved here :", pathImage)
                dataPictures.append([url, brandName, title, src, pathImage, brandFormatted,modelFull, modelFormatted, year, cylindre])
    except Exception as e:
        print("\n \r \n \r \n \r \n \r ERROR HERE !!!!!!!!!!!!!!!!!!!!!!  \n \r \n \r \n \r \n \r",e," !!!!!!!!!!!!!!!!\n \r\n \r")
    finally:
        print("Pass to following one :")
        continue

Brand : honda
Going page : 10
Found : 25 bikes, with 25 brands and 25 models
Looking at a nice Honda cm 125 ... 
Image found !
Image saved here : .\images\honda\cm\honda-cm-honda-cm-125-bleu_159450462.jpg
Looking at a nice Honda xl 1000v varadero ... 
Image found !
Image saved here : .\images\honda\xl v varadero\honda-xl-varadero-1000-jaune_159489805.jpg
Looking at a nice Honda forza 125 ... 
Image found !
Image saved here : .\images\honda\forza\honda-forza-forza-125-2017-noir_159705743.jpg
Looking at a nice Honda sh ... 
Image found !
Image saved here : .\images\honda\sh\honda-sh-honda-sh_159712310.jpg
Looking at a nice Honda cr 80 ... 
Image found !
Image saved here : .\images\honda\cr\honda-cr-honda-slr-650-tracker-scrumbler-cross-noir_159712646.jpg
Looking at a nice Honda sh ... 
Image found !
Image saved here : .\images\honda\sh\honda-sh-honda-sh-rouge_159726196.jpg
Looking at a nice Honda forza 125 ... 
Image found !
Image saved here : .\images\honda\forza\honda-forza-honda-forza

Image found !
Image saved here : .\images\honda\sh\honda-sh-125-inj_157800504.jpg
Looking at a nice Honda africa twin ... 
Image found !
Image saved here : .\images\honda\africa twin\honda-africa-twin-1100-l_157894930.jpg
Looking at a nice Honda xl 500s ... 
Image found !
Image saved here : .\images\honda\xl s\honda-xl-2-honda-500-xls-a-restaurer-avec-lot-de-pieces_157896266.jpg
Looking at a nice Honda cb 1000r ... 
Image found !
Image saved here : .\images\honda\cb r\honda-cb-cb-1000-r-abs_158110898.jpg
Looking at a nice Honda nss ... 
Image found !
Image saved here : .\images\honda\nss\honda-nss-125-a9-f-abs_158347962.jpg
Looking at a nice Honda africa twin ... 
Image found !
Image saved here : .\images\honda\africa twin\honda-africa-twin-1100-l-adventure-sport_158347994.jpg
Looking at a nice Honda gl 1800 goldwing ... 
Image found !
Image saved here : .\images\honda\gl goldwing\honda-gl-1800-airbag_158348028.jpg
Looking at a nice Honda silverwing 125 ... 
Image found !
Image saved h

Image found !
Image saved here : .\images\honda\xl\honda-xl-xlr-600-street-tracker-bleu_158002033.jpg
Looking at a nice Honda nps ... 
Image found !
Image saved here : .\images\honda\nps\honda-nps-scooter-honda-zoomer-noir_158244121.jpg
Looking at a nice Honda sh 300 ... 
Image found !
Image saved here : .\images\honda\sh\honda-sh-honda-sh-300i-gris_158244446.jpg
Looking at a nice Honda cbf 500 ... 
Image found !
Image saved here : .\images\honda\cbf\honda-cbf-honda-cbf-500-gris_158953617.jpg
Looking at a nice Honda lead 110 ... 
Image found !
Image saved here : .\images\honda\lead\honda-lead-honda-lead-110-gris_158953966.jpg
Looking at a nice Honda vision 110 ... 
Image found !
Image saved here : .\images\honda\vision\honda-vision-honda-vision-110-noir_159051365.jpg
Going page : 15
Found : 25 bikes, with 25 brands and 25 models
Looking at a nice Honda xl 600v transalp ... 
Image found !
Image saved here : .\images\honda\xl v transalp\honda-xl-transalp-600-a2-violet_159265827.jpg
Looki

Image found !
Image saved here : .\images\honda\x adv\honda-x-adv-honda-x-adv-2021-noir_159867563.jpg
Looking at a nice Honda pantheon 125 ... 
Image found !
Image saved here : .\images\honda\pantheon\honda-pantheon-honda-fes-pantheon_159896395.jpg
Looking at a nice Honda forza 125 ... 
Image found !
Image saved here : .\images\honda\forza\honda-forza-honda-forza-marron_159896613.jpg
Looking at a nice Honda cb 500 ... 
Image found !
Image saved here : .\images\honda\cb\honda-cb-cb-x-500_159918714.jpg
Looking at a nice Honda forza 300 ... 
Image found !
Image saved here : .\images\honda\forza\honda-forza-honda-forza-300-abs-2019-1ere-main-noir_160028412.jpg
Looking at a nice Honda sh 125 ... 
Image found !
Image saved here : .\images\honda\sh\honda-sh-honda-sh-125-topbox-tablier-parebrise-blanc_160040907.jpg
Looking at a nice Honda cbr 1000f ... 
Image found !
Image saved here : .\images\honda\cbr f\honda-cbr-honda-cbr-rouge_160098068.jpg
Looking at a nice Honda cb 600f hornet ... 
Imag

Image found !
Image saved here : .\images\honda\xl v transalp\honda-xl-transalp-600-noir_158442287.jpg
Looking at a nice Honda forza 125 ... 
Image found !
Image saved here : .\images\honda\forza\honda-forza-vend-forza-125-comme-neuf-noir_158588942.jpg
Looking at a nice Honda gl 1800 goldwing ... 
Image found !
Image saved here : .\images\honda\gl goldwing\honda-gl-goldwing-1800-noir_158639893.jpg
Looking at a nice Honda forza 125 ... 
Image found !
Image saved here : .\images\honda\forza\honda-forza-honda-forza-125-annee-2015-blanc_158691325.jpg
Looking at a nice Honda cb 125r ... 
Image found !
Image saved here : .\images\honda\cb r\honda-cb-honda-cb-125-r-noir_158773163.jpg
Looking at a nice Honda nc 750x ... 
Image found !
Image saved here : .\images\honda\nc x\honda-nc-honda-nc-750-x-dct-2021-blanc_158954966.jpg
Looking at a nice Honda forza 125 ... 
Image found !
Image saved here : .\images\honda\forza\honda-forza-vends-honda-forza-125-2017-noir_158959877.jpg
Looking at a nice Ho

...

Image found !
Image saved here : .\images\yamaha\tmax\yamaha-tmax-t-max-2019-gris-nardo-3400km-gris_158875031.jpg
Looking at a nice Yamaha tmax 560 ... 
Image found !
Image saved here : .\images\yamaha\tmax\yamaha-tmax-tmax-560-anniversary-noir_159001419.jpg
Looking at a nice Yamaha xmax 125 ... 
Image found !
Image saved here : .\images\yamaha\xmax\yamaha-xmax-yamaha-xmax-125-2017-blanc_159153585.jpg
Looking at a nice Yamaha xt 125 ... 
Image found !
Image saved here : .\images\yamaha\xt\yamaha-xt-yamaha-xt-125-de-1983-blanc_159157611.jpg
Looking at a nice Yamaha mt 09 ... 
Image found !
Image saved here : .\images\yamaha\mt\yamaha-mt-mt09-akrapovic-night-fluo-gris_159169446.jpg
Looking at a nice Yamaha xmax 125 ... 
Image found !
Image saved here : .\images\yamaha\xmax\yamaha-xmax-xmax-125-demarrage-sans-cle-toutes-options-revise-et-garantie-gris_159201500.jpg
Looking at a nice Yamaha mt 125 ... 
Image found !
Image saved here : .\images\yamaha\mt\yamaha-mt-mt-125-yamaha-2021-urgent-

Image found !
Image saved here : .\images\yamaha\xsr\yamaha-xsr-xsr-a2-ligne-complete-ixil-black-edition-noir_160032821.jpg
Looking at a nice Yamaha yfm ... 
Image found !
Image saved here : .\images\yamaha\yfm\yamaha-yfm-raptor-700-noir_160076529.jpg
Looking at a nice Yamaha tracer ... 
Image found !
Image saved here : .\images\yamaha\tracer\yamaha-tracer-yamaha-tracer-900-gt-bleu_160080167.jpg
Looking at a nice Yamaha tmax 500 ... 
Image found !
Image saved here : .\images\yamaha\tmax\yamaha-tmax-tmax-500-noir_160082596.jpg
Looking at a nice Yamaha majesty 125 ... 
Image found !
Image saved here : .\images\yamaha\majesty\yamaha-majesty-majesty_160082698.jpg
Looking at a nice Yamaha xmax 125 ... 
Image found !
Image saved here : .\images\yamaha\xmax\yamaha-xmax-xmax-125-abs_160092271.jpg
Looking at a nice Yamaha slider naked ... 
Image found !
Image saved here : .\images\yamaha\slider naked\yamaha-slider-naked-yamaha-slider-50-noir_160124266.jpg
Looking at a nice Yamaha d'elight 125 .

Image found !
Image saved here : .\images\yamaha\nmax\yamaha-nmax-125_158943435.jpg
Going page : 15
Found : 25 bikes, with 25 brands and 25 models
Looking at a nice Yamaha nmax ... 
Image found !
Image saved here : .\images\yamaha\nmax\yamaha-nmax-yamaha-nmax-2021-3400-bleu_158953312.jpg
Looking at a nice Yamaha majesty 400 ... 
Image found !
Image saved here : .\images\yamaha\majesty\yamaha-majesty-yamaha-majesty-400-abs-blanc-gris_158954804.jpg
Looking at a nice Yamaha tricity 125 ... 
Image found !
Image saved here : .\images\yamaha\tricity\yamaha-tricity-scooter-125-abs-yamaha-noir_158959778.jpg
Looking at a nice Yamaha tmax 530 ... 
Image found !
Image saved here : .\images\yamaha\tmax\yamaha-tmax-530_159007352.jpg
Looking at a nice Yamaha xmax 300 ... 
Image found !
Image saved here : .\images\yamaha\xmax\yamaha-xmax-300_159040480.jpg
Looking at a nice Yamaha xmax 125 ... 
Image found !
Default image
Looking at a nice Yamaha tricity ... 
Image found !
Image saved here : .\images\

Image found !
Image saved here : .\images\yamaha\fz\yamaha-fz-yamaha-fz6-s2-noir_160033291.jpg
Looking at a nice Yamaha ch ... 
Image found !
Image saved here : .\images\yamaha\ch\yamaha-ch-rouge_160057459.jpg
Looking at a nice Yamaha xmax ... 
Image found !
Image saved here : .\images\yamaha\xmax\yamaha-xmax-yamaha-x-max-noir_160107652.jpg
Looking at a nice Yamaha xmax ... 
Image found !
Image saved here : .\images\yamaha\xmax\yamaha-xmax-yamaha-x-max-noir_160108401.jpg
Looking at a nice Yamaha mt 07 ... 
Image found !
Image saved here : .\images\yamaha\mt\yamaha-mt-mt07-a2-bleu-electrique-2020-ligne-arrow-encore-garantie-pneus-neufs-noir_160127799.jpg
Looking at a nice Yamaha xmax 125 ... 
Image found !
Image saved here : .\images\yamaha\xmax\yamaha-xmax-xmax-125-abs-noir_152170540.jpg
Looking at a nice Yamaha tdm 850 ... 
Image found !
Image saved here : .\images\yamaha\tdm\yamaha-tdm-vend-850-tdm-yahama-annee-2000_156941627.jpg
Looking at a nice Yamaha tmax ... 
Image found !
Defau

TimeoutException: Message: timeout: Timed out receiving message from renderer: 298.100
  (Session info: headless chrome=95.0.4638.54)
Stacktrace:
Backtrace:
	Ordinal0 [0x009C06F3+2492147]
	Ordinal0 [0x00959BD1+2071505]
	Ordinal0 [0x00862478+1057912]
	Ordinal0 [0x00854CFB+1002747]
	Ordinal0 [0x00853D4D+998733]
	Ordinal0 [0x00854117+999703]
	Ordinal0 [0x0085DBEF+1039343]
	Ordinal0 [0x00867EA9+1081001]
	Ordinal0 [0x0086A53B+1090875]
	Ordinal0 [0x008543F5+1000437]
	Ordinal0 [0x00867D6A+1080682]
	Ordinal0 [0x008B5210+1397264]
	Ordinal0 [0x008A568B+1332875]
	Ordinal0 [0x008821D4+1188308]
	Ordinal0 [0x0088302F+1191983]
	GetHandleVerifier [0x00B467A6+1545030]
	GetHandleVerifier [0x00BF105C+2243580]
	GetHandleVerifier [0x00A4BC97+518199]
	GetHandleVerifier [0x00A4AD80+514336]
	Ordinal0 [0x0095ED2D+2092333]
	Ordinal0 [0x00962EE8+2109160]
	Ordinal0 [0x00963022+2109474]
	Ordinal0 [0x0096CB71+2149233]
	BaseThreadInitThunk [0x766BFA29+25]
	RtlGetAppContainerNamedObjectPath [0x76EC7A9E+286]
	RtlGetAp

In [114]:
#dataPictures = []
#browser.get("https://www.leparking-moto.fr/moto-occasion/royal-enfield.html")
#browser.execute_script("ctrl.set_pageReload(2)")
#thumbnail_results = browser.find_elements(By.XPATH, "//ul[@id='resultats']//section[@class='clearfix complete-holder']//a[@class='external tag_f_titre']")
blockTitleListBrand = browser.find_elements(By.XPATH, "//section[@class='clearfix complete-holder']//div[@class='block-title-list']//span[@class='title-block brand']")
blockTitleListModel = browser.find_elements(By.XPATH, "//section[@class='clearfix complete-holder']//div[@class='block-title-list']//span[@class='sub-title title-block']")
print(len(blockTitleListBrand))
print(len(blockTitleListModel))
print(blockTitleListBrand[0].get_attribute("innerHTML"))
print(blockTitleListModel[0].get_attribute("innerHTML"))
#print("Found "+ str(len(thumbnail_results))+ " ads in page 2")
#res = thumbnail_results[1]
#href = res.get_attribute('href')
#print("Going to : "+str(href))
#browser.get(href)
#img = browser.find_elements(By.XPATH, "//div[@class='fiche-descri clearfix']//img")
#if len(img) == 1:
#    print("Image found !")
#    img = img[0]
#    title = img.get_attribute('title')
#    src = img.get_attribute('src')
#    print("Found image => title = "+str(title)+ " src = "+str(src) )
#    pathImage = saveImage(brandName, title, src)
#    print("Saved here :" pathImage)
#    dataPictures.append([url, brandName, title, src, pathImage])

25
36
Royal enfield
Interceptor


In [42]:
df = pd.DataFrame(dataPictures, columns=['url', 'brandName', 'title',"src", "path", "brandFormatted","modelFull", "modelFormatted", "year"])
print(df)
df.to_csv( os.path.join(CH_DIR,"moto_database.csv"), sep = ";", index=False )

                                                    url brandName  \
0     https://www.leparking-moto.fr/moto-occasion/.h...   triumph   
1     https://www.leparking-moto.fr/moto-occasion/.h...   triumph   
2     https://www.leparking-moto.fr/moto-occasion/.h...   triumph   
3     https://www.leparking-moto.fr/moto-occasion/.h...   triumph   
4     https://www.leparking-moto.fr/moto-occasion/.h...   triumph   
...                                                 ...       ...   
1700  https://www.leparking-moto.fr/moto-occasion/.h...       bmw   
1701  https://www.leparking-moto.fr/moto-occasion/.h...       bmw   
1702  https://www.leparking-moto.fr/moto-occasion/.h...       bmw   
1703  https://www.leparking-moto.fr/moto-occasion/.h...       bmw   
1704  https://www.leparking-moto.fr/moto-occasion/.h...       bmw   

                                       title  \
0                       TRIUMPH STREET TWIN    
1               TRIUMPH TIGER TIGER EXPLORER   
2      TRIUMPH BONNEVILLE B

In [75]:
#Install driver
url_db = "https://www.autoevolution.com/moto/"
opts=webdriver.ChromeOptions()
opts.headless=True
driver_database = webdriver.Chrome(ChromeDriverManager().install() ,options=opts)
print("Going page : "+str(url_db))
driver_database.get(url_db)
resultBrands = [(a.get_attribute('href'), a.get_attribute("title")) for a in  driver_database.find_elements(By.XPATH, "//div[@class='col2width fl bcol-white carman']/a[1]") ]

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/95.0.4638.54/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\avuat\.wdm\drivers\chromedriver\win32\95.0.4638.54]
C:\Users\avuat\AppData\Local\Temp/ipykernel_5452/4184361449.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_database = webdriver.Chrome(ChromeDriverManager().install() ,options=opts)


Going page : https://www.autoevolution.com/moto/


In [97]:
#distinctMakes = list(set([m for (m, _,_) in allModels]))
#resultBrands = [(href, rb) for href, rb in  resultBrands  if rb not in distinctMakes]
allModels = []
allGens = []
for href, make in resultBrands:
    print("Going at make :",make )
    driver_database.get(href)
    resultsModel = [(make, a.get_attribute('href'), a.get_attribute("title")) for a in  driver_database.find_elements(By.XPATH, "//div[@class='col2width bcol-white fl']/a[1]") ]
    allModels += resultsModel
    for _, href_model, model in resultsModel:
        print("Going at model :",model )
        driver_database.get(href_model)
        resultsYears = [(make, model, a.get_attribute('href'), 
                         a.get_attribute("title").replace(" specs, photos and information", "").strip(" ")) 
                        for a 
                        in  driver_database.find_elements(By.XPATH, "//div[@class='col1width fl']/h2/a") ]
        allGens += resultsYears

Going at make : HARLEY DAVIDSON
Going at model : HARLEY DAVIDSON Electra Glide
Going at model : HARLEY DAVIDSON Dyna
Going at model : HARLEY DAVIDSON Trike
Going at model : HARLEY DAVIDSON FREEWHEELER
Going at model : HARLEY DAVIDSON Street Glide
Going at model : HARLEY DAVIDSON Ultra
Going at model : HARLEY DAVIDSON Sportster
Going at model : HARLEY DAVIDSON Street
Going at model : HARLEY DAVIDSON CVO
Going at model : HARLEY DAVIDSON Road Glide
Going at model : HARLEY DAVIDSON Road King
Going at model : HARLEY DAVIDSON Softail
Going at model : HARLEY DAVIDSON Firefighter
Going at model : HARLEY DAVIDSON Peace Officer
Going at model : HARLEY DAVIDSON Police
Going at model : HARLEY DAVIDSON Sport Glide
Going at model : HARLEY DAVIDSON FORTY-EIGHT
Going at model : HARLEY DAVIDSON Pan America
Going at model : HARLEY DAVIDSON SUPERLOW
Going at model : HARLEY DAVIDSON FXDR
Going at model : HARLEY DAVIDSON IRON
Going at model : HARLEY DAVIDSON CLE
Going at model : HARLEY DAVIDSON Fire/Rescue

Going at model : ITALJET Torpedo
Going at model : ITALJET Jet Set
Going at model : ITALJET CAFE RACER
Going at model : ITALJET Millennium
Going at model : ITALJET Grifon 900
Going at model : ITALJET VELOCIFERO
Going at make : IZH
Going at model : IZH 350
Going at model : IZH 49
Going at model : IZH 56
Going at model : IZH Planeta 5
Going at make : JAWA
Going at model : JAWA 125
Going at model : JAWA 250
Going at model : JAWA 350
Going at model : JAWA 50
Going at model : JAWA 500
Going at model : JAWA 650
Going at model : JAWA 660
Going at make : JUNAK
Going at model : JUNAK M11
Going at model : JUNAK NK
Going at model : JUNAK M16
Going at model : JUNAK M25
Going at model : JUNAK M20
Going at model : JUNAK S
Going at model : JUNAK B20
Going at model : JUNAK M07
Going at model : JUNAK M07-R
Going at model : JUNAK M10
Going at model : JUNAK WB
Going at make : KANUNI
Going at model : KANUNI Motorun
Going at model : KANUNI Tigrina 100
Going at model : KANUNI Bobcat
Going at model : KANUNI S

Going at model : MBK Flipper
Going at model : MBK Stunt Naked
Going at model : MBK Flame X
Going at model : MBK Skycruiser 125
Going at model : MBK Ovetto
Going at model : MBK X-Limit Supermotard
Going at model : MBK Booster
Going at model : MBK Motobecane
Going at make : MEGELLI
Going at model : MEGELLI NAKED
Going at model : MEGELLI MOTARD
Going at model : MEGELLI SPORT
Going at make : MIDUAL
Going at model : MIDUAL TYPE 1
Going at model : MIDUAL Roadster 900
Going at make : MIKILON
Going at model : MIKILON PENTORA
Going at model : MIKILON A6
Going at model : MIKILON CD
Going at model : MIKILON D
Going at model : MIKILON GT
Going at make : MODENAS
Going at model : MODENAS DINAMIK
Going at model : MODENAS ACE
Going at model : MODENAS CT
Going at model : MODENAS EFI
Going at model : MODENAS CERIA
Going at model : MODENAS CTRIC
Going at model : MODENAS ELEGAN
Going at model : MODENAS GT
Going at model : MODENAS JAGUH
Going at model : MODENAS KARISMA
Going at model : MODENAS KRISS
Going 

Going at model : OSSA Super Pioneer 250
Going at model : OSSA Urbe
Going at make : PETRONAS
Going at model : PETRONAS FP1
Going at make : PEUGEOT
Going at model : PEUGEOT Speedfight
Going at model : PEUGEOT XP6
Going at model : PEUGEOT Elystar
Going at model : PEUGEOT Vivacity
Going at model : PEUGEOT Geopolis
Going at model : PEUGEOT BLASTER
Going at model : PEUGEOT Looxor
Going at model : PEUGEOT V-Clic
Going at model : PEUGEOT Ludix
Going at model : PEUGEOT VOGS2
Going at model : PEUGEOT Satelis
Going at model : PEUGEOT XR6
Going at model : PEUGEOT Elyseo
Going at model : PEUGEOT PEUGEOT
Going at make : PGO
Going at model : PGO G-MAX
Going at model : PGO BUBU
Going at model : PGO B1
Going at model : PGO B2
Going at model : PGO BIG MAX
Going at model : PGO EVO G-MAX
Going at make : PIAGGIO
Going at model : PIAGGIO Typhoon
Going at model : PIAGGIO BV
Going at model : PIAGGIO Fly 50/150
Going at model : PIAGGIO MP3
Going at model : PIAGGIO Typhoon 50
Going at model : PIAGGIO X10
Going 

Going at model : TRIUMPH Sprint
Going at model : TRIUMPH Tiger 900
Going at model : TRIUMPH TR
Going at model : TRIUMPH Trident
Going at model : TRIUMPH TSS
Going at model : TRIUMPH TSX
Going at model : TRIUMPH TT 600
Going at model : TRIUMPH X 75 Hurricane
Going at make : URAL
Going at model : URAL M70
Going at model : URAL Patrol
Going at model : URAL Gear-Up
Going at model : URAL T
Going at model : URAL Tourist
Going at model : URAL Solo
Going at model : URAL Ranger
Going at model : URAL Wolf 750
Going at model : URAL Cross
Going at model : URAL Limited Edition
Going at model : URAL Retro
Going at model : URAL Sportsman
Going at model : URAL Troyka
Going at model : URAL Wolf
Going at make : VELOCETTE
Going at model : VELOCETTE D
Going at model : VELOCETTE K
Going at model : VELOCETTE KTT MK
Going at model : VELOCETTE L.E. Mk
Going at model : VELOCETTE MAC
Going at model : VELOCETTE MAF
Going at model : VELOCETTE MOV
Going at model : VELOCETTE MSS
Going at model : VELOCETTE MT
Going 

Going at model : YAMAHA TZR
Going at model : YAMAHA V-MAX
Going at model : YAMAHA Vino 125
Going at model : YAMAHA Warrior
Going at model : YAMAHA WR 450 F
Going at model : YAMAHA XS
Going at model : YAMAHA XT 225
Going at model : YAMAHA XT 350
Going at model : YAMAHA XZ 550
Going at model : YAMAHA YA
Going at model : YAMAHA YD
Going at model : YAMAHA YFA 125
Going at model : YAMAHA YFM
Going at model : YAMAHA YR
Going at make : ZERO
Going at model : ZERO DS
Going at model : ZERO DSP
Going at model : ZERO DSR
Going at model : ZERO FX
Going at model : ZERO S
Going at model : ZERO SR
Going at model : ZERO FXP
Going at model : ZERO FXS
Going at model : ZERO XU
Going at make : ZUNDAPP
Going at model : ZUNDAPP 200 S
Going at model : ZUNDAPP Bella
Going at model : ZUNDAPP K
Going at model : ZUNDAPP TROPHY
Going at model : ZUNDAPP Z


In [98]:
print("Nombre de modèles : ", len(allModels))
print("Nombre de modèles et générations : ", len(allGens))
df = pd.DataFrame(allGens, columns=['make', 'model', 'href',"full name"])

Nombre de modèles :  1249
Nombre de modèles et générations :  9176


In [99]:
sort_gens = sorted(Counter(df["full name"]).items(), key=lambda x: x[1], reverse=True)
sort_models = sorted(Counter(df["model"]).items(), key=lambda x: x[1], reverse=True)
print(sort_gens[0:20])
print(sort_models[0:20])

[('HARLEY DAVIDSON Electra Glide Classic', 34), ('HARLEY DAVIDSON Heritage Softail Classic', 33), ('HARLEY DAVIDSON Super Glide', 31), ('HARLEY DAVIDSON Street Glide', 31), ('HARLEY DAVIDSON Low Rider', 26), ('HARLEY DAVIDSON Fat Boy', 26), ('HARLEY DAVIDSON Electra Glide', 25), ('HARLEY DAVIDSON Electra Glide Ultra Classic', 23), ('HARLEY DAVIDSON Sportster 883', 22), ('HARLEY DAVIDSON Road King', 22), ('HONDA CB 400SF Super Four', 20), ('HARLEY DAVIDSON Wide Glide', 17), ('HARLEY DAVIDSON 1200 Custom', 17), ('HARLEY DAVIDSON Road King Classic', 17), ('SUZUKI DR-Z400S', 17), ('HONDA CB 600F Hornet', 16), ('HONDA CB 125T', 16), ('SUZUKI GSX-R1000', 16), ('SUZUKI GSX-R600', 16), ('SUZUKI GSX-R750', 16)]
[('HONDA CB', 430), ('HARLEY DAVIDSON Sportster', 213), ('HARLEY DAVIDSON Softail', 176), ('HONDA CBR', 153), ('YAMAHA V Star', 125), ('HARLEY DAVIDSON Dyna', 124), ('SUZUKI GSX-R', 112), ('HARLEY DAVIDSON Electra Glide', 107), ('SUZUKI Boulevard', 96), ('HONDA Shadow', 92), ('HONDA CRF'

In [69]:
resultBrands

[('https://www.autoevolution.com/moto/adler/', 'Adler'),
 ('https://www.autoevolution.com/moto/aeon/', 'AEON'),
 ('https://www.autoevolution.com/moto/aermacchi/', 'AERMACCHI'),
 ('https://www.autoevolution.com/moto/ajp/', 'AJP'),
 ('https://www.autoevolution.com/moto/ajs/', 'AJS'),
 ('https://www.autoevolution.com/moto/alta-motors/', 'ALTA MOTORS'),
 ('https://www.autoevolution.com/moto/amazonas/', 'AMAZONAS'),
 ('https://www.autoevolution.com/moto/aprilia/', 'APRILIA'),
 ('https://www.autoevolution.com/moto/arctic-cat/', 'ARCTIC CAT'),
 ('https://www.autoevolution.com/moto/ariel/', 'ARIEL'),
 ('https://www.autoevolution.com/moto/atk/', 'ATK'),
 ('https://www.autoevolution.com/moto/bajaj/', 'BAJAJ'),
 ('https://www.autoevolution.com/moto/barigo/', 'BARIGO'),
 ('https://www.autoevolution.com/moto/benelli/', 'BENELLI'),
 ('https://www.autoevolution.com/moto/beta/', 'BETA'),
 ('https://www.autoevolution.com/moto/bfg/', 'BFG'),
 ('https://www.autoevolution.com/moto/big-bear-choppers/',
  '